In [10]:
import numpy_financial as npf

class ProjectFinance:
    """
    A class to calculate NPV, IRR, PI, Payback, and Discounted Payback for a project.
    """
    def __init__(self, name, cash_flows, rate):
        self.name = name
        self.cash_flows = cash_flows
        self.initial_investment = abs(cash_flows[0]) # Stored as positive cost
        self.future_cash_flows = cash_flows[1:]
        self.rate = rate
        
    def calculate_npv(self):
        """Calculates Net Present Value (NPV)."""
        return npf.npv(self.rate, self.cash_flows)

    def calculate_irr(self):
        """Calculates Internal Rate of Return (IRR)."""
        return npf.irr(self.cash_flows)

    def calculate_pi(self):
        """Calculates Profitability Index (PI)."""
        # Calculate the PV of future cash inflows (CF1 through CF4)
        pv_inflows = npf.npv(self.rate, [0] + self.future_cash_flows)
        
        if self.initial_investment == 0:
            return float('inf')
        return pv_inflows / self.initial_investment

    def calculate_dcf(self):
        """
        Calculates the Present Value (Discounted Cash Flow) for each year's inflow.
        Returns a list of discounted future cash flows (excluding Year 0).
        """
        dcf_list = []
        for t, cf in enumerate(self.future_cash_flows, start=1):
            dcf = cf / ((1 + self.rate) ** t)
            dcf_list.append(dcf)
        return dcf_list
    
    def calculate_payback_period(self):
        """Calculates the standard Payback Period."""
        remaining_cost = self.initial_investment
        cumulative_cf = 0
        
        for t, cf in enumerate(self.future_cash_flows, start=1):
            cumulative_cf += cf
            if cumulative_cf >= self.initial_investment:
                fraction = (remaining_cost / cf)
                return t - 1 + fraction
            remaining_cost -= cf
        return float('inf') # Never pays back

    def calculate_discounted_payback_period(self): # FIX: Removed 'rate' argument from definition
        """Calculates the Discounted Payback Period."""
        # Now correctly calls calculate_dcf() without passing self.rate as an argument
        dcf_list = self.calculate_dcf() 
        
        remaining_cost = self.initial_investment
        cumulative_dcf = 0
        
        for t, dcf in enumerate(dcf_list, start=1):
            cumulative_dcf += dcf
            if cumulative_dcf >= self.initial_investment:
                fraction = (remaining_cost / dcf)
                return t - 1 + fraction
            remaining_cost -= dcf
        return float('inf') # Never pays back

# --- Project Data ---
project_s_flows = [-500, 150, 200, 250]
project_l_flows = [-300, 100, 120, 140]
project_m_flows = [-400, 120, 160, 180]

# --- Projects Initialized with their Specific Discount Rates ---
project_s = ProjectFinance("Project S", project_s_flows, 0.08) # Rate: 8%
project_l = ProjectFinance("Project L", project_l_flows, 0.10) # Rate: 10%
project_m = ProjectFinance("Project M", project_m_flows, 0.07) # Rate: 7%

# --- Results ---
results = []
for project in [project_s, project_l, project_m]:
    npv = project.calculate_npv()
    irr = project.calculate_irr()
    pi = project.calculate_pi()
    pb = project.calculate_payback_period()
    dpb = project.calculate_discounted_payback_period()
    dcf_list = project.calculate_dcf()
    
    # Format DCF list for output, creating a clean string field
    dcf_output = ", ".join([f'Y{i+1}: ${dcf:.2f}' for i, dcf in enumerate(dcf_list)])
    
    results.append({
        "Project": project.name,
        "NPV": f"${npv:,.2f}",
        "IRR": f"{irr * 100:.2f}%",
        "PI": f"{pi:.3f}",
        "Payback Period": f"{pb:.2f} years",
        "Discounted Payback Period": f"{dpb:.2f} years",
        "Yearly DCF": dcf_output
    })

# --- Output ---
print("--- Project Financial Metrics Summary ---")
for result in results:
    print(f"\n{result['Project']} (Discount Rate: {float(result['IRR'].strip('%')) / (float(result['IRR'].strip('%')) * (100 / float(result['IRR'].strip('%')))) * 100:.0f}%):")
    print(f"  NPV: {result['NPV']}")
    print(f"  IRR: {result['IRR']}")
    print(f"  PI:  {result['PI']}")
    print(f"  Payback Period: {result['Payback Period']}")
    print(f"  Discounted Payback: {result['Discounted Payback Period']}")
    print(f"  Yearly DCF: {result['Yearly DCF']}")

--- Project Financial Metrics Summary ---

Project S (Discount Rate: 9%):
  NPV: $8.81
  IRR: 8.90%
  PI:  1.018
  Payback Period: 2.60 years
  Discounted Payback: 2.96 years
  Yearly DCF: Y1: $138.89, Y2: $171.47, Y3: $198.46

Project L (Discount Rate: 9%):
  NPV: $-4.73
  IRR: 9.15%
  PI:  0.984
  Payback Period: 2.57 years
  Discounted Payback: inf years
  Yearly DCF: Y1: $90.91, Y2: $99.17, Y3: $105.18

Project M (Discount Rate: 7%):
  NPV: $-1.17
  IRR: 6.85%
  PI:  0.997
  Payback Period: 2.67 years
  Discounted Payback: inf years
  Yearly DCF: Y1: $112.15, Y2: $139.75, Y3: $146.93
